# Single Event with Complex Analysis

## Part One (Data Exploration)

#### Prerequisites
This notebook is designed to work with a Stroom server process running on `localhost`, into which data from `EventGen` application has been ingested and indexed in the manner described in `stroom-analytic-demo`.

You must set the environmental variable `STROOM_API_KEY` to the API token associated with a suitably privileged Stroom user account before starting the Jupyter notebook server process.

In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import from_json, col, coalesce, unix_timestamp,lit,to_timestamp,hour,date_format,date_trunc
from IPython.display import display
import time,os,json

#### Schema Discovery
It is necessary to specify the structure of the JSON data arriving on the topic.  This structure can be determined at runtime.

As the same format of data is also available via an indexed search using the `stroom-spark-datasource`, one way to determine the JSON schema is by interrogating the data held in the `Sample Index` Stroom index.

The specified pipeline is a Stroom Search Extraction Pipeline that uses the stroom:json XSLT function to create a JSON representation of the entire event.  This field is called "Json" by default but the name of the field that contains the JSON representation can (optionally) be changed with the parameter jsonField.

In this manner, all data is returned as a single JSON structure within the field **json**

In [2]:
spark = SparkSession \
    .builder \
    .appName("MyTestApp") \
    .getOrCreate()

schemaDf = spark.read.format('stroom.spark.datasource.StroomDataSource').load(
        token=os.environ['STROOM_API_KEY'],host='localhost',protocol='http',
        uri='api/stroom-index/v2',
        index='fc38ca93-cdc0-4c50-94e5-6445e92fc376',
        pipeline='13143179-b494-4146-ac4b-9a6010cada89',
        maxResults='100000').filter((col('idxEventTime') > '2019-11-01T00:00:00.000Z'))
json_schema = spark.read.json(schemaDf.rdd.map(lambda row: row.json)).schema

json_schema

KeyError: 'STROOM_API_KEY'

In [11]:
output = json_schema.json()
output

'{"fields":[{"metadata":{},"name":"EventDetail","nullable":true,"type":{"fields":[{"metadata":{},"name":"Authenticate","nullable":true,"type":{"fields":[{"metadata":{},"name":"Action","nullable":true,"type":"string"},{"metadata":{},"name":"Outcome","nullable":true,"type":{"fields":[{"metadata":{},"name":"Permitted","nullable":true,"type":"string"},{"metadata":{},"name":"Reason","nullable":true,"type":"string"},{"metadata":{},"name":"Success","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"User","nullable":true,"type":{"fields":[{"metadata":{},"name":"Id","nullable":true,"type":"string"}],"type":"struct"}}],"type":"struct"}},{"metadata":{},"name":"Process","nullable":true,"type":{"fields":[{"metadata":{},"name":"Action","nullable":true,"type":"string"},{"metadata":{},"name":"Command","nullable":true,"type":"string"},{"metadata":{},"name":"Type","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"TypeId","nullable":true,"type":"string"}],

In [13]:
schemaFile = open("event-logging-v3.2.3.json", "w")
schemaFile.write(json.dumps(json.loads(output), indent=4, sort_keys=True))
schemaFile.close()

The generated schema file can be used by `stateMonitor` application.